In [3]:
import sys
import mysql.connector
from mysql.connector import errorcode
import time
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import csv

In [7]:
try:
    conn = mysql.connector.connect(host = 'kurly-db.cxt53rg7mbem.ap-northeast-2.rds.amazonaws.com',port = 3306,database = 'kurlyDB',user = 'root',password = 'zjfflxnejans')
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor(prepared=True)
    #물품(1-122)
    order_num = 3
    for i in range(1003,1005):
        sql_address = "SELECT address_name FROM address where address_no = (%s);"
        cursor.execute(sql_address,[i])
        for result in cursor:
            address_name = result[0]
        print(address_name)
        #상온
        ordinary_storage_weight = 0
        ordinary_storage_volume = 0
        ordinary_storage_boxes = 0
        #냉장
        cold_storage_weight = 0
        cold_storage_volume = 0
        cold_storage_boxes = 0
        #냉동
        freeze_storage_weight = 0
        freeze_storage_volume = 0
        freeze_storage_boxes = 0
        #총
        boxes_volume = 10000
        total_boxes = 0
        total_weight = 0
        order_total_quantity = random.randint(2,10)
        
        order_object_no= []
        for i in range(1,order_total_quantity):
            object_no = random.randint(1,122)
            object_quantity = random.randint(1,3)
            print("object no:",object_no,"object_quantity",object_quantity)
            sql = "SELECT object_shape,object_storage,object_weight,object_volume FROM object where object_no = (%s);"
            cursor.execute(sql,[object_no])
            for result in cursor:
                if result[0] == 2:
                    total_boxes += (1 * object_quantity)
                    total_weight += (result[2] * object_quantity)
                else:
                    if result[1] == 0:
                        cold_storage_weight += (result[2] * object_quantity)
                        cold_storage_volume += (result[3] * object_quantity)

                    elif result[1] == 1:
                        freeze_storage_weight += (result[2] * object_quantity)
                        freeze_storage_volume += (result[3] * object_quantity)

                    else:
                        ordinary_storage_weight += (result[2] * object_quantity)
                        ordinary_storage_volume += (result[3] * object_quantity)
        if cold_storage_volume != 0:                
            cold_storage_boxes += (cold_storage_volume // boxes_volume) + 1
        if freeze_storage_volume != 0:
            freeze_storage_boxes += (freeze_storage_volume // boxes_volume) + 1
        if ordinary_storage_volume !=0:
            ordinary_storage_boxes += (ordinary_storage_volume // boxes_volume) + 1
        
        total_boxes = total_boxes + cold_storage_boxes + freeze_storage_boxes + ordinary_storage_boxes
        total_weight = total_weight + cold_storage_weight + freeze_storage_weight + ordinary_storage_weight
        print(total_boxes,"box",total_weight,"g")

Connection established
서울 강남구 영동대로 216 하나은행 학여울역지점
object no: 96 object_quantity 2
1 box 1000 g
서울 강남구 영동대로 50길 10 엠솔학원
object no: 65 object_quantity 2
object no: 35 object_quantity 2
object no: 5 object_quantity 1
object no: 36 object_quantity 2
object no: 86 object_quantity 3
object no: 55 object_quantity 3
object no: 92 object_quantity 2
5 box 15660 g


In [5]:
def delivery_difficulty2(total_boxes,total_weight,elevator_exist, delivery_floor):
    box_w = 0.6
    weight_w = 0.0003 
    elevator_y_w = 1
    elevator_n_w = 1
    floor_w = 0.15
    
    average_object_w = 3.5
    difficulty_w = 1
    #우체국 배송 운임 참고 기준
    weight_changed = ((total_weight//4000) +1)
    if weight_changed >= total_boxes:
        object_total_w = weight_changed
    else:
        object_total_w = total_boxes
    object_final_w = object_total_w / average_object_w
    
    #엘리베이터 존재 X
    if elevator_exist == 0:
        difficulty_total = object_final_w * (elevator_n_w + (delivery_floor - 1) * floor_w)
    
    #엘리베이터 존재 O
    else:
        difficulty_total = object_final_w * elevator_y_w 
    
    deli_difficulty = difficulty_total / difficulty_w
    if deli_difficulty <1:
        deli_difficulty = 1
    elif deli_difficulty>2.5:
        deli_difficulty = 2.5
    return object_final_w,deli_difficulty

In [161]:
elevator = 0
floor = 5
test1 = delivery_difficulty2(total_boxes, total_weight, elevator, floor)
print(test1)

(1.1428571428571428, 1.8285714285714285)
